In [1]:
%pip install pandas langchain-community langchain unidecode nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
from unidecode import unidecode
from nltk.corpus import stopwords

# baixa as stopwords em pt-br
stopwords_list = set(stopwords.words('portuguese'))

def normalize_text(text):
    # deixa tudo minúsculo
    text = text.lower()
    
    text = unidecode(text) # remove os acentos codificados

    # remove espaços desnecessários
    text = re.sub(r'\s+', ' ', text).strip()
    
    # remove pontuação
    text = re.sub(r'[^\w\s]', '', text)
    
    return text

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords_list]) # remove as palavras do texto que estiverem na lista de stopwords

In [12]:
import os
import pandas as pd
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# define as colunas do csv que serão usadas
useful_data = ['product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'overall_rating', 'review_text']

# carrega o csv
def load_data(file_path):
    # lê o arquivo csv
    df = pd.read_csv(file_path)
    
    # exclui as colunas que não serão usadas
    df_reduced = df.drop(columns=[col for col in df.columns if col not in useful_data])

    # limpa e normaliza o texto
    for column in useful_data:
        df_reduced[column] = df_reduced[column].apply(lambda x: normalize_text(str(x)))
        df_reduced[column] = df_reduced[column].apply(lambda x: remove_stopwords(str(x)))

    # define o nome e o caminho do arquivo de saída
    result_file_name = 'data_processed.csv'  # nome do arquivo de saída
    new_file_path = os.path.join(r'src/data', result_file_name)  # caminho para salvar o arquivo de saída na pasta 'data'
    
    # salva os dados em csv
    df_reduced.to_csv(new_file_path, index=False)

    return new_file_path

def load_and_chunk(file_path):
    # gera os documentos com o csv processado
    loader = CSVLoader(file_path=file_path, encoding='utf-8', csv_args={
        'delimiter': ',', 'quotechar': '"', 'fieldnames': useful_data
    }) # classe do langchain para manipular os dados
    # delimiter define o que separa as colunas do csv (no caso, uma vírgula)
    # quotechar define o caractere que envolve strings, permitindo que uma string tenha vírgulas sem ser divididas em colunas
    # fieldnames são os nomes das colunas que serão carregadas

    docs = loader.load() # retorna os documentos gerados

    # chunkeniza e adiciona overlap nos dados
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
    splits = text_splitter.split_documents(docs)

    return splits

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

def generate_embeddings(splits):
    # inicia a classe do langchain para geração dos embeddings
    print("Inicializando o modelo de embeddings...")
    
    hf = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2',  # modelo da sentence transformers voltado para pt-br para a geração de embeddings
        model_kwargs={'device': 'cpu'},  # define o dispositivo que está sendo usado (CPU, GPU etc)
        encode_kwargs={'normalize_embeddings': False}  # define se os embeddings devem ser normalizados (False mantém a escala original)
    )
    
    print("Modelo de embeddings inicializado com sucesso.")
    
    # cria um vetor de armazenamento a partir dos documentos fornecidos e do modelo de embeddings utilizando a classe chroma do langchain
    print(f"Gerando embeddings para {len(splits)} documentos...")
    
    vectorstore = Chroma.from_documents(
        documents=splits,  # documentos que foram divididos em partes menores
        embedding=hf  # modelo de embeddings utilizado para transformar os documentos em vetores
    )

    print("Embeddings gerados e armazenados com sucesso.")
    
    # retorna o vetor de armazenamento como um retriever (objeto) para consultas posteriores
    retriever = vectorstore.as_retriever()
    print("Retorno do vetor de armazenamento como retriever concluído.")
    
    return retriever


In [18]:
# Bloco de execução condicional
if __name__ == "__main__":
    file_path = r'H:\Github\Fatec\atmchat\src\data\chat_data.csv'  # substitua pelo caminho correto do seu arquivo CSV
    new_file_path = load_data(file_path)  # Carrega e processa os dados
    splits = load_and_chunk(new_file_path)  # Gera os chunks a partir do CSV processado

    # Gerar embeddings a partir dos chunks
    print("Iniciando a geração de embeddings com dados de exemplo...")
    retriever = generate_embeddings(splits)  # Chame a função com a variável splits já definida
    print("Geração de embeddings finalizada.")

    # Imprimir a coleção de embeddings
    print("Coleção de embeddings gerada com sucesso.")

    # Exemplo de consulta
    query = "Smartphone"  # Insira a consulta que deseja realizar
    print(f"\nRealizando consulta: '{query}'")

    # Ajuste aqui conforme a versão do langchain que você está utilizando
    results = retriever.get_relevant_documents(query)  # Realiza a consulta

    # Exibir resultados
    print(f"Número de resultados encontrados: {len(results)}")
    for i, result in enumerate(results):
        print(f"Resultado {i + 1}:")
        print(result)  # Aqui você pode imprimir o resultado encontrado


Iniciando a geração de embeddings com dados de exemplo...
Inicializando o modelo de embeddings...
Modelo de embeddings inicializado com sucesso.
Gerando embeddings para 125 documentos...
Embeddings gerados e armazenados com sucesso.
Retorno do vetor de armazenamento como retriever concluído.
Geração de embeddings finalizada.
Coleção de embeddings gerada com sucesso.

Realizando consulta: 'Smartphone'
Número de resultados encontrados: 4
Resultado 1:
page_content='product_name: smartphone samsung galaxy s7 edge android 60 tela 55 128gb 4g camera 12mp black piano
product_brand: samsung
site_category_lv1: celulares smartphones
site_category_lv2: smartphone
overall_rating: 5
review_text: melhor custo beneficio exatamente anunciado recomendo' metadata={'row': 38, 'source': 'src/data\\data_processed.csv'}
Resultado 2:
page_content='product_name: smartphone samsung galaxy s7 edge android 60 tela 55 128gb 4g camera 12mp black piano
product_brand: samsung
site_category_lv1: celulares smartphones